##Coverletter

cover letters will be inputed and summarised into less than 150 words with key information like
1. role category
2. skills
3. experience level correlated with skill
4. significant experiences

In [1]:
import numpy as np
import pandas as pd

In [2]:
#reading csv for the data
from google.colab import drive
drive.mount('/content/drive')

#combining the train test split as we do not have to train
filepath_train = '/content/drive/MyDrive/Colab Notebooks/TalentFit/final_coverletter_df_train.csv'
filepath_test = '/content/drive/MyDrive/Colab Notebooks/TalentFit/final_coverletter_df_test.csv'
df_train = pd.read_csv(filepath_train)
df_test = pd.read_csv(filepath_test)

combined_df = pd.concat([df_train, df_test], axis=0)
combined_df.reset_index(inplace=True, drop=True)

Mounted at /content/drive


In [3]:
#setting up openai
!pip install openai
from google.colab import userdata
from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [4]:
#picking out longer resumes to form a set of example coverletters to test the summary function on

combined_df = combined_df.sort_values(by='Cover Letter', key=lambda x: x.str.len(), ascending=False)
# Set the display.max_colwidth option to -1 to display full text
pd.set_option('display.max_colwidth', None)

# Print the full text of the 'Cover Letter' column
test_cl = pd.DataFrame(combined_df['Cover Letter'].head().unique())

test_cl

,0
0,"Dear Hiring Manager,\n\nI am writing to express my interest in the Data Scientist position at XYZ Analytics Solutions. With my strong background in data analysis and experience in implementing digital analytics solutions, I believe I would be a valuable asset to your team.\n\nIn my current role as a Data Scientist at XYZ Analytics Solutions, I have successfully built out the process and templates for Adobe Audience Manager implementations, and trained team members to effectively use them. I have a deep understanding of the digital analytics space and have architectured and implemented digital analytics solutions for various websites and mobile applications. Additionally, I have extensive experience in customizing digital analytics solutions within the Adobe Analytics Cloud.\n\nI am highly skilled in implementing digital marketing, analytics, and other tags through Tag Management Systems, utilizing my advanced JavaScript skills. I have also gained expertise in data science, including data classification, data mining, and forecasting using machine learning capabilities in Adobe Analytics. Furthermore, I am proficient in querying data pipelines, analyzing and visualizing data with tools like Tableau, and designing tests to optimize user experience and business outcomes.\n\nWith my strong programming skills in Java, SQL, R, Python, and experience building data pipelines, I am confident in my ability to support the development team through the coding process and ensure successful implementation as outlined in the technical instructions.\n\nThank you for considering my application. I am excited about the opportunity to contribute to XYZ Analytics Solutions and further enhance my skills in the data science field. I look forward to discussing how my qualifications align with your company's needs.\n\nSincerely,\nJohn Smith"
1,"Dear Hiring Manager,\n\nI am writing to express my interest in the Senior Data Scientist position at XYZ Technologies. With over 7 years of experience in the field, I am confident in my ability to contribute to your Data Science & AI CoE and drive innovation in the ATI industry.\n\nThroughout my career, I have gained strong expertise in deep learning, CUDA/TensorRT, and modern data compression. I have successfully delivered high-quality research, innovation, and engineering projects on time, always striving for excellence. My hands-on experience with OpenCV, SOD, and other computer vision libraries has allowed me to develop state-of-the-art solutions and integrate computer vision into industrial-level products.\n\nI have a strong background in biometrics, machine learning, and deep learning, and I am well-versed in Agile methodologies. My proficiency in machine learning frameworks such as TensorFlow, Keras, Pytorch, and Caffe has enabled me to develop cutting-edge solutions. Additionally, my published research in computer vision, image processing, and machine learning showcases my commitment to advancing the field.\n\nWith a Master's degree in Computer Science specializing in AI/Computer Vision, I possess a solid foundation in advanced mathematics, image/video processing, computer vision, and AI/machine learning theories.\n\nI am excited about the opportunity to join XYZ Technologies and contribute to the company's mission of revolutionizing the ATI industry with Data Science AI Technology. I am eager to learn from and collaborate with the talented data science and engineering community at XYZ Technologies.\n\nThank you for considering my application. I look forward to the possibility of discussing how my skills and qualifications align with your requirements.\n\nSincerely,\nJohn Smith"
2,"Dear Hiring Manager,\n\nI am writing to express my interest in the Data Scientist position at XYZ Analytics. With my strong background in Data Science and Machine Learning, I believe I am wellsuited for this role.\n\nIn my current position as a Data Scientist at XYZ Corporation, I have successfully led analytic approaches

##prompt function: 'summarise'

This function automates the process of analyzing and summarizing cover letters specifically tailored for individuals in the information technology industry.

It uses OpenAI's GPT-3.5-turbo model to interpret the content of a cover letter and identify the most fitting technical role.

Additionally, it extracts and summarizes the top five relevant skills along with their experience levels and the top three significant professional experiences related to the assigned role.

###Parameters:

cover_letter (string): The text of the cover letter to be analyzed.
Returns:

summary (string): A concise summary formatted to include the determined role, top skills with experience levels, and significant professional experiences. The summary is designed to be brief, not exceeding 150 words (~200 tokens).

###Operational Details:

Role Determination: The function processes the input text to match the cover letter with one appropriate IT-related position.

Skill and Experience Extraction:
Utilizes natural language processing to identify and rank the top five skills mentioned in the cover letter, assessing the associated experience level for each.

Significant Experience Highlighting:
Extracts three key professional experiences that are most relevant to the identified role, ensuring these instances are specific and quantifiable.
Output Formatting:

The output is structured to present the analysis in a clear, easy-to-read format, emphasizing clarity and precision in the summary.

####Note:

The function assumes that the input cover letter is well-formatted and sufficiently detailed to extract meaningful insights. The quality of the summary may vary with the detail and clarity of the cover letter provided.


In [5]:
def summarise(coverletter):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": """
        You will be provided with a cover letter from an individual in the information technology industry. Your task is to analyze the content of the cover letter and determine which of the following tech roles it best aligns with:
        ["Software Developer", "Systems Administrator", "Project Manager", "Web Developer", "Database Administrator", "Java Developer", "Python Developer", "Security Analyst", "Network Administrator"].

        Assign only one role from the list provided, based on the skills and experiences mentioned in the cover letter. Then, summarize the top 5 skills and the corresponding experience level, along with the top 3 most significant experiences directly related to the assigned role. Please ensure the summary is concise, not exceeding 150 words in total.

        Present your analysis in the following format:
        "Role: [Chosen Role]
        Skills and Experience Level:
        1. [Skill 1], [Experience Level];
        2. [Skill 2], [Experience Level]; ...
        Significant Experiences:
        1. [Experience 1];
        2. [Experience 2]; ..."

        Note: The skills and experiences should be listed in order of relevance and impact as demonstrated in the cover letter. Make sure the experiences are specific and quantifiable, not generic statements. If any skills or experiences do not fit into the specified roles, they should not be included in the summary.
        """
        },
        {"role": "user", "content":coverletter}
      ],
      temperature=0.3,    # More deterministic & factual (0.2 ~ 0.5)
      max_tokens=200,     # ~150 words
      top_p=1,            # Full probability distribution
      frequency_penalty=0.5,  # Reduce repetition
      presence_penalty=0.5,   # Encourage diversity
    )
    return response.choices[0].message.content

####example outputs/usage example

In [6]:
#output as a df
cl_summary_examples = pd.DataFrame(columns=["input","output"])

#apply the summary function on coverletters in the input df
for i in range(len(test_cl)):
  input = test_cl[0][i]
  output = summarise(test_cl[0][i]) #applying the function
  cl_summary_examples.loc[i] = [input,output]

cl_summary_examples.head()

,input,output
0,"Dear Hiring Manager,\n\nI am writing to express my interest in the Data Scientist position at XYZ Analytics Solutions. With my strong background in data analysis and experience in implementing digital analytics solutions, I believe I would be a valuable asset to your team.\n\nIn my current role as a Data Scientist at XYZ Analytics Solutions, I have successfully built out the process and templates for Adobe Audience Manager implementations, and trained team members to effectively use them. I have a deep understanding of the digital analytics space and have architectured and implemented digital analytics solutions for various websites and mobile applications. Additionally, I have extensive experience in customizing digital analytics solutions within the Adobe Analytics Cloud.\n\nI am highly skilled in implementing digital marketing, analytics, and other tags through Tag Management Systems, utilizing my advanced JavaScript skills. I have also gained expertise in data science, including data classification, data mining, and forecasting using machine learning capabilities in Adobe Analytics. Furthermore, I am proficient in querying data pipelines, analyzing and visualizing data with tools like Tableau, and designing tests to optimize user experience and business outcomes.\n\nWith my strong programming skills in Java, SQL, R, Python, and experience building data pipelines, I am confident in my ability to support the development team through the coding process and ensure successful implementation as outlined in the technical instructions.\n\nThank you for considering my application. I am excited about the opportunity to contribute to XYZ Analytics Solutions and further enhance my skills in the data science field. I look forward to discussing how my qualifications align with your company's needs.\n\nSincerely,\nJohn Smith","Role: Data Scientist\nSkills and Experience Level:\n1. Data Analysis, Expert;\n2. Digital Analytics Solutions Implementation, Expert;\n3. Tag Management Systems, Advanced;\n4. Machine Learning, Advanced;\n5. Data Visualization with Tableau, Proficient.\nSignificant Experiences:\n1. Built process and templates for Adobe Audience Manager implementations;\n2. Architectured and implemented digital analytics solutions for websites and mobile applications;\n3. Customized digital analytics solutions within the Adobe Analytics Cloud."
1,"Dear Hiring Manager,\n\nI am writing to express my interest in the Senior Data Scientist position at XYZ Technologies. With over 7 years of experience in the field, I am confident in my ability to contribute to your Data Science & AI CoE and drive innovation in the ATI industry.\n\nThroughout my career, I have gained strong expertise in deep learning, CUDA/TensorRT, and modern data compression. I have successfully delivered high-quality research, innovation, and engineering projects on time, always striving for excellence. My hands-on experience with OpenCV, SOD, and other computer vision libraries has allowed me to develop state-of-the-art solutions and integrate computer vision into industrial-level products.\n\nI have a strong background in biometrics, machine learning, and deep learning, and I am well-versed in Agile methodologies. My proficiency in machine learning frameworks such as TensorFlow, Keras, Pytorch, and Caffe has enabled me to develop cutting-edge solutions. Additionally, my published research in computer vision, image processing, and machine learning showcases my commitment to advancing the field.\n\nWith a Master's degree in Computer Science specializing in AI/Computer Vision, I possess a solid foundation in advanced mathematics, image/video processing, computer vision, and AI/machine learning theories.\n\nI am excited about the opportunity to join XYZ Technologies and contribute to the company's mission of revolutionizing the ATI industry with Data Science AI Technology. I am eager to learn from and collaborate with the talented data science and enginee

In [13]:
#single example output
print(cl_summary_examples['output'][1])

Role: Senior Data Scientist
Skills and Experience Level:
1. Deep Learning, Expert;
2. Machine Learning Frameworks (TensorFlow, Keras, Pytorch, Caffe), Expert;
3. Computer Vision, Expert;
4. Agile Methodologies, Advanced;
5. Research Publication, Advanced.

Significant Experiences:
1. Led and delivered multiple high-quality research and engineering projects in deep learning and computer vision.
2. Published research in computer vision, image processing, and machine learning.
3. Master's degree in Computer Science specializing in AI/Computer Vision.
